# Create lineage based label images

In this notebook we provide code to create label images where cells are colored by track id.

In [1]:
import numpy as np
import pandas as pd
import pathlib
import delta
import tifffile


## Set paths

In [2]:
proj_dir = pathlib.Path(pathlib.Path.home(), 'I2ICourse', 'Project2B')
pos_file = proj_dir / 'ProcessedData' / 'Position000000.pkl' # path to .pkl file (output from Delta)
df_file = proj_dir / 'ProcessedData' / 'Position000000_dataframe.csv' # path to .csv file containing lineage info (output from Delta post-processing)

## Define function

In [ ]:
def convert_label_image(label_stack, df, frame, property='id_cell'):
    """
    Plot spatial map of cell property
    
    Parameters:
        label_stack: Numpy ND array [t,y,x] with label image stack
        df: pandas data frame of lineage object
        frame: frame to show
        property: key of cell property contained in lineage object
    Returns:
        spatial_map: Numpy ND array [y,x] spatial map of cell property at given frame
    """

    # get label image:
    labels = label_stack[frame,:,:] #assumes frame is the first dimension
    spatial_map = np.zeros_like(labels)
    
    # Go over cells in selected frame:
    for cnb in np.unique(labels):
        if cnb==0: continue
        #assign cells mask area the phenotype of choice
        #note: Delta id_seg uses zero based indexing, label image 1 based indexing!
        spatial_map[labels==cnb] = df.loc[(df['frames']==frame) & (df['id_seg']==cnb-1),property].item()
    
    return spatial_map

## Load Data

In [4]:
#create postion object
pos = delta.pipeline.Position(None,None,None)
pos.load(pos_file) #load position file
df = pd.read_csv(df_file) #load lineage dataframe

#create label stack
labels = pos.rois[0].label_stack
labelstack = np.stack(labels, axis=0) #load label data from position file

## Process data

In [5]:
#loop over all frames
labelim_new_list = [convert_label_image(labelstack, df, frame=fr) for fr in range(len(labels))]
labels_new = np.stack(labelim_new_list) #create 3D array from list of 2D arrays

### Write Tiff Files

In [ ]:
for i in range(labels_new.shape[0]):
    tifffile.imwrite(f"label_im_{i}.tiff", labels_new[i, :, :])